# Active Learning Example

**Note:** This code utilizes functions from an older version of the Label Studio SDK (v0.0.34). While the newer versions v1.0 and above still support the functionalities of the old version (see `label_studio_sdk._legacy` for reference), we recommend using the latest Label Studio SDK v1.0 or higher.

Active learning is a branch of machine learning that seeks to minimize the total amount of data required for labeling by strategically sampling data that provides insight into the problem you're trying to solve so that you can focus on labeling that data.

Follow this example to write a Python script using the [Label Studio SDK](https://labelstud.io/sdk/index.html) that performs active learning with a text classification machine learning model. 

## Set up the connection

In [ ]:
!pip install label-studio-sdk

Start by configuring the connection to the Label Studio API. You can retrieve your API key from your user profile in Label Studio. In your script, write the following, replacing the **LABEL_STUDIO_URL** and **LABEL_STUDIO_API_KEY** with your own: 

In [ ]:
import os
LABEL_STUDIO_URL = os.getenv('LABEL_STUDIO_URL', 'http://localhost:8080')
LABEL_STUDIO_API_KEY = os.getenv('LABEL_STUDIO_API_KEY')

Then, import the Client module from the Label Studio SDK to make sure that you successfully connected to the API:

In [ ]:
from label_studio_sdk.client import LabelStudio

ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=LABEL_STUDIO_API_KEY)

## Create a project

After connecting to the Label Studio API with the SDK, create a project in Label Studio to perform active learning with your data labeling tasks. This project performs sentiment analysis for a passage of text. See the [sentiment analysis template](https://labelstud.io/templates/sentiment_analysis.html) for more. 

In [ ]:
env_pid = os.getenv('LABEL_STUDIO_PROJECT_ID')
if env_pid:
    project = ls.projects.get(id=int(env_pid))
else:
    project = ls.projects.create(
        title='AL Project Created from SDK - DEMO',
        label_config='''
        <View>
        <Text name="text" value="$text"/>
        <Choices name="sentiment" toName="text" choice="single" showInLine="true">
            <Choice value="Positive"/>
            <Choice value="Negative"/>
            <Choice value="Neutral"/>
        </Choices>
        </View>
        '''
    )
project

In an active learning scenario, you want to label the tasks with the lowest machine learning model prediction scores first. You can set up **uncertainty sampling** for your tasks to automatically reorder tasks by prediction score, from low to high.

In [ ]:
# Some OSS versions don't support sampling updates; skip this step in the notebook

## Set up an example machine learning model

This examples uses a simple TF-IDF Text Classification model built on the [`scikit-learn` API](https://scikit-learn.org/stable/). To perform active learning with this model, we must be able to retrain model weights and make inferences.

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

labels_map = {
    'Positive': 0,
    'Negative': 1,
    'Neutral': 2
}
inv_labels_map = {idx: label for label, idx in labels_map.items()}


def get_model():
    # Initialize model with random weights
    return make_pipeline(TfidfVectorizer(), LogisticRegression(C=10, verbose=True))


def train_model(model, input_texts, output_labels):
    # Train the model, given a list of input texts and output labels
    model.fit(input_texts, [labels_map[label] for label in output_labels])


def get_model_predictions(model, input_texts):
    # Make model inference and return predicted labels and associated prediction scores
    probabilities = model.predict_proba(input_texts)
    predicted_label_indices = np.argmax(probabilities, axis=1)
    predicted_scores = probabilities[np.arange(len(predicted_label_indices)), predicted_label_indices]
    return [inv_labels_map[i] for i in predicted_label_indices], predicted_scores

## Perform active learning

Collect the annotated tasks from Label Studio so that you can use them to train the model. Each task is stored in [Label Studio JSON format](https://labelstud.io/guide/export.html#Label-Studio-JSON-format-of-annotated-tasks), with `"text"` field used as input and `"choices"` annotation field to store output label:

In [ ]:
# Fetch labeled tasks using v2 tasks API
all_tasks_full = list(ls.tasks.list(project=project.id, fields='all'))
_labeled = [t for t in all_tasks_full if getattr(t, 'annotations', None)]
texts, labels = [], []
if not _labeled:
    print(f'No labeled tasks found in project "{project.title}" (id={project.id}).')
    PROCEED_AL = False
else:
    for t in _labeled:
        data_dict = t.data if isinstance(t.data, dict) else getattr(getattr(t, 'data', {}), 'model_dump', lambda: {} )()
        texts.append(data_dict.get('text', ''))
        # use the first annotation's first choice
        first_ann = t.annotations[0]
        res = first_ann.get('result') if isinstance(first_ann, dict) else getattr(first_ann, 'result', None)
        if res:
            labels.append(res[0]['value']['choices'][0])
    if not texts:
        print('No text content found in labeled tasks.')
        PROCEED_AL = False
    else:
        print(f'Found {len(texts)} annotated texts and {len(set(labels))} classes')
        PROCEED_AL = True

Update the model weights based on the annotated tasks. We use `"train_model()"` function from example machine learning described above, but in principle it could be any other classifier trainer.

In [ ]:
if not PROCEED_AL:
    print('No labeled data; skipping training and prediction steps.')
else:
    model = get_model()
    train_model(model, texts, labels)

After collecting the annotated tasks, collect the unlabeled tasks so that the machine learning model can make predictions. Because there can be a large number of unlabeled tasks, you can sample them and retrieve only a small subset of data. In this case, collect a random sample of 100 unlabeled tasks:

In [ ]:
import random
if not PROCEED_AL:
    _unlabeled_batch = []
else:
    all_tasks = list(ls.tasks.list(project=project.id, fields='task_only'))
    annotated_ids = {t.id for t in _labeled}
    unlabeled_tasks = [t for t in all_tasks if t.id not in annotated_ids]
    if not unlabeled_tasks:
        print('No unlabeled tasks to predict; skipping prediction step.')
        _unlabeled_batch = []
        PROCEED_AL = False
    else:
        batch_ids = random.sample([t.id for t in unlabeled_tasks], min(100, len(unlabeled_tasks)))
        batch_map = {t.id: t for t in all_tasks}
        _unlabeled_batch = [batch_map[i] for i in batch_ids]

With the subset of unlabeled tasks that you collected, you can make model inferences to get the predictions from the text classification model:

In [ ]:
import numpy as np
if not PROCEED_AL:
    pred_labels, pred_scores = [], []
    print('Skipping predictions because training was skipped or no unlabeled tasks.')
else:
    texts = [task.data.get('text', '') for task in _unlabeled_batch]
    pred_labels, pred_scores = get_model_predictions(model, texts)

    # Example:
    for i, (text, labels, scores) in enumerate(zip(texts, pred_labels, pred_scores)):
        print(f'{text} --> {labels} [{scores}]')
        print('===============================')

## Send predictions to Label Studio

After the model makes its predictions, return the predictions to Label Studio so that annotators can review and update them. 

Define a model version to identify the latest batch of predictions, in this example based on the amount of data used to retrain the model, but you can use any arbitrary unique name. Setting model version is optional step, but in Active Learning scenario, it helps you to control which model to show in the next iteration.

In [ ]:
if PROCEED_AL:
    model_version = f'model_{len(texts)}'
    print(model_version)
else:
    model_version = None
    print('Skipping model version update.')

Format the predictions and add them to each task:

In [ ]:
if PROCEED_AL and _unlabeled_batch:
    for task, pred_label, pred_score in zip(_unlabeled_batch, pred_labels, pred_scores):
        ls.predictions.create(
            task=task.id,
            result=[{
                'from_name': 'sentiment',
                'to_name': 'text',
                'type': 'choices',
                'value': {
                    'choices': [pred_label]
                }
            }],
            score=float(pred_score),
            model_version=model_version
        )
    print(f'{len(pred_labels)} tasks have been preannotated with model predictions')
else:
    print('Skipping sending predictions.')

Lastly, update the Label Studio settings to use the newly-created model version when performing uncertainty sampling and displaying pre-annotated tasks to annotators:

In [ ]:
if PROCEED_AL and model_version:
    ls.projects.update(id=project.id, model_version=model_version)
else:
    print('Skipping project update.')

In [ ]:
print('Project updated.')

That's it. Now you can open your project page, check **Predictions** and **Prediction score** column 

## Conclusion

That's it! The Label Studio SDK makes creating an active learning loop that is easily repeatable with a script. You can run this script on a regular cadence, or use [Label Studio Webhooks](https://labelstud.io/guide/webhooks.html) to perform event-driven active learning. See more about [active learning in Label Studio](https://labelstud.io/guide/active_learning.html).